# Echo Programming Workflow Example

This notebook demonstrates how to use the Echo programming tools for:
1. Generating Echo transfer CSVs
2. Creating minimap TSV for demultiplexing
3. Generating heatmap mappings for visualization

## 1. Generate Echo Transfer CSV

In [ ]:
from echo_transfer import generate_primer_transfer_csv

# Generate a transfer CSV for 96 primers from a 96-well source to a 384-well destination
generate_primer_transfer_csv(
    num_primers=96,
    source_plate_format=96,
    dest_plate_format=384,
    transfer_volume_nl=100.0,
    output_file='primer_transfer.csv'
)

# Preview the output
import pandas as pd
df = pd.read_csv('primer_transfer.csv')
print(f"Generated {len(df)} transfers")
df.head()

## 2. Generate Minimap TSV for Demultiplexing

In [ ]:
from generate_minimap_tsv import generate_minimap_tsv, generate_fasta

# Generate TSV from barcode CSV
generate_minimap_tsv(
    input_csv='example_barcodes.csv',
    output_tsv='barcodes_minimap.tsv',
    include_well=False
)

# Also generate FASTA format
generate_fasta(
    input_csv='example_barcodes.csv',
    output_fasta='barcodes.fasta',
    include_well=True
)

# Preview the TSV output
df = pd.read_csv('barcodes_minimap.tsv', sep='\t', header=None, names=['Barcode', 'Sequence'])
df.head()

## 3. Generate Heatmap Mapping

In [ ]:
from generate_heatmap_mapping import generate_heatmap_mapping, generate_plate_layout_matrix

# Generate coordinate mapping
generate_heatmap_mapping(
    input_csv='example_barcodes.csv',
    output_csv='heatmap_coordinates.csv'
)

# Generate plate layout matrix for direct plotting
generate_plate_layout_matrix(
    input_csv='example_barcodes.csv',
    output_csv='plate_layout.csv',
    plate_format=384
)

# Preview the coordinate mapping
df = pd.read_csv('heatmap_coordinates.csv')
df.head()

## 4. Visualize Plate Layout as Heatmap

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Read the coordinate mapping
coords = pd.read_csv('heatmap_coordinates.csv')

# Create a mock data set (replace with your demultiplexing results)
# For example, this could be read counts per barcode
coords['ReadCount'] = np.random.randint(100, 10000, len(coords))

# Create a 16x24 matrix for 384-well plate (adjust for 96-well: 8x12)
plate_matrix = np.zeros((16, 24))

for _, row in coords.iterrows():
    plate_matrix[row['Row_Index'], row['Column_Index']] = row['ReadCount']

# Plot heatmap
fig, ax = plt.subplots(figsize=(14, 8))
im = ax.imshow(plate_matrix, cmap='viridis', aspect='auto')

# Set ticks and labels
ax.set_xticks(range(24))
ax.set_yticks(range(16))
ax.set_xticklabels(range(1, 25))
ax.set_yticklabels([chr(65+i) for i in range(16)])

ax.set_xlabel('Column')
ax.set_ylabel('Row')
ax.set_title('384-Well Plate Read Count Heatmap')

# Add colorbar
plt.colorbar(im, ax=ax, label='Read Count')

plt.tight_layout()
plt.show()

## Clean up temporary files

In [ ]:
import os

# Remove temporary files
for f in ['primer_transfer.csv', 'barcodes_minimap.tsv', 'barcodes.fasta', 
          'heatmap_coordinates.csv', 'plate_layout.csv']:
    if os.path.exists(f):
        os.remove(f)
        print(f"Removed {f}")